In [ ]:
import boto3
import quilt3

In [ ]:
%env AWS_PROFILE=staging

In [ ]:
bucket = "quilt-bio-staging"

In [ ]:
all_packages = quilt3.list_packages(f"s3://{bucket}")

In [ ]:
maybe_problematic = [p for p in all_packages if p.startswith("odd")]

In [ ]:
maybe_problematic

In [ ]:
s3 = boto3.client("s3")
problematic = []
max_size_bytes = 20_000_000 = 20MB # adjust as needed, size in bytes
for p in maybe_problematic:
    for time, hash_ in quilt3.list_package_versions(p, f"s3://{bucket}"):
        key = f".quilt/packages/{hash_}"
        head = s3.head_object(Bucket=bucket_name, Key=key)
        size = head.get("ContentLength")
        if size > max_size_bytes:
            problematic.append({
                "Bucket": bucket,
                "Package": p,
                "Key": key,
                "Size": size,
                "TopHash": hash_
            })
problematic

# Caution: real DELETE happens next

In [ ]:
# delete from Quilt package revisions
for p in problematic:
    quilt3.delete_package(
        p['Package'],
        registry=f"s3://{p['Bucket']}",
        top_hash=p['TopHash']
    )

In [ ]:
# deep delete manifest from S3
for p in problematic:
    s3.delete_object(
        Bucket=bucket,
        Key=p['Key']
    )